In [ ]:
using Flux
using Zygote
using Printf

In [253]:
function normal(val::Float64)
    σ = 0.05 * val
    μ = val
    return σ * randn() + μ
end

normal(7.)

6.972855376649661

In [254]:
function forward_model_euler(state::Vector{Float64}, Vm::Float64)
    dt::Float64 = 0.001

    Rm = normal(8.4)
    kt = normal(0.042)
    km = normal(0.042)
    mr = normal(0.095)
    Lr = normal(0.085)
    Jr = mr * Lr ^ 2 / 12
    Dr = normal(0.00027)
    mp = normal(0.024)
    Lp = normal(0.129)
    Jp = mp * Lp ^ 2 / 12
    Dp = normal(0.00005)
    g = normal(9.81)
    θ, α, θ̇, α̇ = state
    
#     Rm::Float64 = 8.4
#     kt::Float64 = 0.042
#     km::Float64 = 0.042
#     mr::Float64 = 0.095
#     Lr::Float64 = 0.085
#     Jr::Float64 = mr * Lr ^ 2 / 12
#     Dr::Float64 = 0.00027
#     mp::Float64 = 0.024
#     Lp::Float64 = 0.129
#     Jp::Float64 = mp * Lp ^ 2 / 12
#     Dp::Float64 = 0.00005
#     g::Float64 = 9.81

    tau::Float64 = -(km * (Vm - km * θ̇)) / Rm  # torque

    # From Rotary Pendulum Workbook
    θ̈::Float64 = (-Lp*Lr*mp*(-8.0 *Dp*α̇ + Lp^2*mp*θ̇^2*sin(2.0 *α) 
            + 4.0 *Lp*g*mp*sin(α))*cos(α) + (4.0 *Jp + Lp^2*mp)*(4.0 *Dr*θ̇ 
            + Lp^2*α̇*mp*θ̇*sin(2.0 *α) + 2.0 *Lp*Lr*α̇^2*mp*sin(α) 
            - 4.0 *tau))/(4.0 *Lp^2*Lr^2*mp^2*cos(α)^2 - (4.0 *Jp 
            + Lp^2*mp)*(4.0 *Jr + Lp^2*mp*sin(α)^2 + 4.0 *Lr^2*mp))
    α̈::Float64 = (2.0 *Lp*Lr*mp*(4.0 *Dr*θ̇ + Lp^2*α̇*mp*θ̇*sin(2.0 *α) 
            + 2.0 *Lp*Lr*α̇^2*mp*sin(α) - 4.0 *tau)*cos(α) - 0.5*(4.0 *Jr 
            + Lp^2*mp*sin(α)^2 + 4.0 *Lr^2*mp)*(-8.0 *Dp*α̇ + Lp^2*mp*θ̇^2*sin(2.0 *α) 
            + 4.0 *Lp*g*mp*sin(α)))/(4.0 *Lp^2*Lr^2*mp^2*cos(α)^2 
            - (4.0 *Jp + Lp^2*mp)*(4.0 *Jr + Lp^2*mp*sin(α)^2 + 4.0 *Lr^2*mp))

    θ̇ += θ̈ * dt
    α̇ += α̈ * dt

    θ += θ̇ * dt
    α += α̇ * dt

    θ = ((θ + π) % (2 * π)) - π
    α = ((α + π) % (2 * π)) - π

    [θ, α, θ̇, α̇]
end

forward_model_euler (generic function with 5 methods)

In [255]:
let

state, Vm = [0.0, 0.0, 0.1, 0.0], 0.0
    
println(state)
@printf("θ=%.4f, α=%.4f, θ̇=%.4f, α̇=%.4f\n", state[1], state[2], state[3], state[4])
    
for i in 1:20000
    state = forward_model_euler(state, Vm)
    
    if i % 10 == 0
        @printf("θ=%.4f, α=%.4f, θ̇=%.4f, α̇=%.4f\n", state[1], state[2], state[3], state[4])
    end

end
    
end

[0.0, 0.0, 0.1, 0.0]
θ=0.0000, α=0.0000, θ̇=0.1000, α̇=0.0000
θ=0.0010, α=-0.0000, θ̇=0.0994, α̇=-0.0006
θ=0.0020, α=-0.0000, θ̇=0.0989, α̇=-0.0012
θ=0.0030, α=-0.0000, θ̇=0.0982, α̇=-0.0019
θ=0.0040, α=-0.0000, θ̇=0.0976, α̇=-0.0025
θ=0.0049, α=-0.0001, θ̇=0.0969, α̇=-0.0033
θ=0.0059, α=-0.0001, θ̇=0.0962, α̇=-0.0040
θ=0.0068, α=-0.0002, θ̇=0.0955, α̇=-0.0048
θ=0.0078, α=-0.0002, θ̇=0.0946, α̇=-0.0059
θ=0.0087, α=-0.0003, θ̇=0.0936, α̇=-0.0071
θ=0.0097, α=-0.0004, θ̇=0.0926, α̇=-0.0084
θ=0.0106, α=-0.0005, θ̇=0.0914, α̇=-0.0100
θ=0.0115, α=-0.0006, θ̇=0.0904, α̇=-0.0116
θ=0.0124, α=-0.0007, θ̇=0.0889, α̇=-0.0137
θ=0.0133, α=-0.0008, θ̇=0.0874, α̇=-0.0161
θ=0.0141, α=-0.0010, θ̇=0.0856, α̇=-0.0187
θ=0.0150, α=-0.0012, θ̇=0.0836, α̇=-0.0219
θ=0.0158, α=-0.0015, θ̇=0.0813, α̇=-0.0257
θ=0.0166, α=-0.0017, θ̇=0.0786, α̇=-0.0300
θ=0.0174, α=-0.0021, θ̇=0.0754, α̇=-0.0351
θ=0.0181, α=-0.0024, θ̇=0.0719, α̇=-0.0409
θ=0.0188, α=-0.0029, θ̇=0.0678, α̇=-0.0479
θ=0.0195, α=-0.0034, θ̇=0.0632, α̇=

θ=-0.7801, α=-2.0850, θ̇=1.9688, α̇=-10.0438
θ=-0.7562, α=-2.1927, θ̇=2.7540, α̇=-11.3962
θ=-0.7231, α=-2.3146, θ̇=3.7949, α̇=-12.8401
θ=-0.6780, α=-2.4520, θ̇=5.1459, α̇=-14.5067
θ=-0.6172, α=-2.6073, θ̇=6.8847, α̇=-16.4100
θ=-0.5366, α=-2.7834, θ̇=9.0425, α̇=-18.5743
θ=-0.4352, α=-2.9800, θ̇=10.9845, α̇=-20.4564
θ=-0.3204, α=-3.1890, θ̇=11.5431, α̇=-20.9311
θ=-0.2116, α=-3.3912, θ̇=10.1267, α̇=-19.4209
θ=-0.1232, α=-3.5724, θ̇=7.7470, α̇=-17.0445
θ=-0.0578, α=-3.7304, θ̇=5.6521, α̇=-14.8308
θ=-0.0103, α=-3.8680, θ̇=4.0823, α̇=-12.9547
θ=0.0234, α=-3.9887, θ̇=2.8454, α̇=-11.3564
θ=0.0466, α=-4.0944, θ̇=1.9202, α̇=-9.9345
θ=0.0619, α=-4.1862, θ̇=1.2287, α̇=-8.5930
θ=0.0712, α=-4.2652, θ̇=0.7137, α̇=-7.3415
θ=0.0761, α=-4.3317, θ̇=0.3232, α̇=-6.0838
θ=0.0777, α=-4.3860, θ̇=0.0297, α̇=-4.8836
θ=0.0767, α=-4.4283, θ̇=-0.1909, α̇=-3.6985
θ=0.0739, α=-4.4589, θ̇=-0.3528, α̇=-2.5507
θ=0.0696, α=-4.4781, θ̇=-0.4790, α̇=-1.4100
θ=0.0642, α=-4.4858, θ̇=-0.5872, α̇=-0.2569
θ=0.0578, α=-4.4820, θ

θ=-0.5647, α=-3.2661, θ̇=1.6655, α̇=-2.9337
θ=-0.5493, α=-3.2933, θ̇=1.4376, α̇=-2.5414
θ=-0.5363, α=-3.3163, θ̇=1.1803, α̇=-2.0892
θ=-0.5261, α=-3.3345, θ̇=0.8875, α̇=-1.6050
θ=-0.5187, α=-3.3478, θ̇=0.6050, α̇=-1.1003
θ=-0.5143, α=-3.3560, θ̇=0.3135, α̇=-0.5760
θ=-0.5127, α=-3.3589, θ̇=0.0239, α̇=-0.0623
θ=-0.5140, α=-3.3567, θ̇=-0.2541, α̇=0.4531
θ=-0.5180, α=-3.3494, θ̇=-0.5198, α̇=0.9641
θ=-0.5246, α=-3.3371, θ̇=-0.7796, α̇=1.4282
θ=-0.5338, α=-3.3203, θ̇=-1.0259, α̇=1.8777
θ=-0.5453, α=-3.2994, θ̇=-1.2486, α̇=2.2734
θ=-0.5590, α=-3.2745, θ̇=-1.4584, α̇=2.6395
θ=-0.5745, α=-3.2465, θ̇=-1.6250, α̇=2.9267
θ=-0.5915, α=-3.2160, θ̇=-1.7486, α̇=3.1392
θ=-0.6094, α=-3.1838, θ̇=-1.8219, α̇=3.2653
θ=-0.6278, α=-3.1509, θ̇=-1.8434, α̇=3.3046
θ=-0.6461, α=-3.1180, θ̇=-1.8075, α̇=3.2524
θ=-0.6637, α=-3.0862, θ̇=-1.7141, α̇=3.1046
θ=-0.6801, α=-3.0564, θ̇=-1.5746, α̇=2.8751
θ=-0.6949, α=-3.0292, θ̇=-1.3931, α̇=2.5764
θ=-0.7077, α=-3.0054, θ̇=-1.1861, α̇=2.2274
θ=-0.7183, α=-2.9852, θ̇=-0.9511

θ=-0.6709, α=-3.1258, θ̇=0.2156, α̇=-0.4033
θ=-0.6686, α=-3.1300, θ̇=0.2324, α̇=-0.4343
θ=-0.6662, α=-3.1345, θ̇=0.2428, α̇=-0.4541
θ=-0.6638, α=-3.1391, θ̇=0.2466, α̇=-0.4617
θ=-0.6613, α=-3.1437, θ̇=0.2431, α̇=-0.4570
θ=-0.6590, α=-3.1482, θ̇=0.2337, α̇=-0.4413
θ=-0.6567, α=-3.1525, θ̇=0.2171, α̇=-0.4141
θ=-0.6547, α=-3.1564, θ̇=0.1938, α̇=-0.3772
θ=-0.6529, α=-3.1599, θ̇=0.1662, α̇=-0.3310
θ=-0.6514, α=-3.1629, θ̇=0.1346, α̇=-0.2772
θ=-0.6502, α=-3.1654, θ̇=0.0985, α̇=-0.2175
θ=-0.6494, α=-3.1672, θ̇=0.0612, α̇=-0.1525
θ=-0.6490, α=-3.1684, θ̇=0.0220, α̇=-0.0827
θ=-0.6491, α=-3.1688, θ̇=-0.0202, α̇=-0.0112
θ=-0.6495, α=-3.1685, θ̇=-0.0609, α̇=0.0614
θ=-0.6503, α=-3.1675, θ̇=-0.0983, α̇=0.1286
θ=-0.6515, α=-3.1659, θ̇=-0.1352, α̇=0.1927
θ=-0.6530, α=-3.1636, θ̇=-0.1669, α̇=0.2506
θ=-0.6548, α=-3.1608, θ̇=-0.1955, α̇=0.3017
θ=-0.6569, α=-3.1576, θ̇=-0.2206, α̇=0.3449
θ=-0.6593, α=-3.1539, θ̇=-0.2402, α̇=0.3790
θ=-0.6617, α=-3.1500, θ̇=-0.2530, α̇=0.4024
θ=-0.6643, α=-3.1459, θ̇=-0.259

θ=-0.7103, α=-3.1450, θ̇=-0.0077, α̇=-0.0041
θ=-0.7104, α=-3.1450, θ̇=-0.0128, α̇=0.0046
θ=-0.7105, α=-3.1449, θ̇=-0.0177, α̇=0.0133
θ=-0.7107, α=-3.1447, θ̇=-0.0223, α̇=0.0213
θ=-0.7110, α=-3.1445, θ̇=-0.0265, α̇=0.0288
θ=-0.7113, α=-3.1441, θ̇=-0.0303, α̇=0.0357
θ=-0.7116, α=-3.1437, θ̇=-0.0334, α̇=0.0412
θ=-0.7119, α=-3.1433, θ̇=-0.0358, α̇=0.0456
θ=-0.7123, α=-3.1428, θ̇=-0.0375, α̇=0.0488
θ=-0.7127, α=-3.1423, θ̇=-0.0385, α̇=0.0508
θ=-0.7131, α=-3.1418, θ̇=-0.0388, α̇=0.0516
θ=-0.7135, α=-3.1413, θ̇=-0.0383, α̇=0.0509
θ=-0.7138, α=-3.1408, θ̇=-0.0372, α̇=0.0489
θ=-0.7142, α=-3.1403, θ̇=-0.0352, α̇=0.0457
θ=-0.7145, α=-3.1399, θ̇=-0.0325, α̇=0.0412
θ=-0.7149, α=-3.1395, θ̇=-0.0293, α̇=0.0358
θ=-0.7151, α=-3.1392, θ̇=-0.0257, α̇=0.0295
θ=-0.7154, α=-3.1389, θ̇=-0.0218, α̇=0.0226
θ=-0.7156, α=-3.1387, θ̇=-0.0175, α̇=0.0152
θ=-0.7157, α=-3.1386, θ̇=-0.0130, α̇=0.0074
θ=-0.7158, α=-3.1386, θ̇=-0.0086, α̇=-0.0002
θ=-0.7159, α=-3.1387, θ̇=-0.0040, α̇=-0.0082
θ=-0.7159, α=-3.1388, θ̇=0.00

θ=-0.7336, α=-3.1414, θ̇=-0.0003, α̇=-0.0066
θ=-0.7336, α=-3.1415, θ̇=-0.0001, α̇=-0.0069
θ=-0.7336, α=-3.1416, θ̇=-0.0001, α̇=-0.0070
θ=-0.7336, α=-3.1416, θ̇=-0.0001, α̇=-0.0069
θ=-0.7336, α=-3.1417, θ̇=-0.0003, α̇=-0.0066
θ=-0.7336, α=-3.1418, θ̇=-0.0005, α̇=-0.0062
θ=-0.7336, α=-3.1418, θ̇=-0.0009, α̇=-0.0056
θ=-0.7337, α=-3.1419, θ̇=-0.0013, α̇=-0.0048
θ=-0.7337, α=-3.1419, θ̇=-0.0018, α̇=-0.0039
θ=-0.7337, α=-3.1420, θ̇=-0.0024, α̇=-0.0029
θ=-0.7337, α=-3.1420, θ̇=-0.0029, α̇=-0.0019
θ=-0.7338, α=-3.1420, θ̇=-0.0035, α̇=-0.0009
θ=-0.7338, α=-3.1420, θ̇=-0.0041, α̇=0.0001
θ=-0.7338, α=-3.1420, θ̇=-0.0047, α̇=0.0012
θ=-0.7339, α=-3.1420, θ̇=-0.0052, α̇=0.0021
θ=-0.7339, α=-3.1420, θ̇=-0.0057, α̇=0.0031
θ=-0.7340, α=-3.1419, θ̇=-0.0062, α̇=0.0039
θ=-0.7341, α=-3.1419, θ̇=-0.0066, α̇=0.0047
θ=-0.7341, α=-3.1418, θ̇=-0.0070, α̇=0.0054
θ=-0.7342, α=-3.1418, θ̇=-0.0072, α̇=0.0058
θ=-0.7343, α=-3.1417, θ̇=-0.0074, α̇=0.0061
θ=-0.7344, α=-3.1416, θ̇=-0.0074, α̇=0.0062
θ=-0.7344, α=-3.1416

θ=-0.7425, α=-3.1416, θ̇=-0.0022, α̇=0.0008
θ=-0.7425, α=-3.1416, θ̇=-0.0023, α̇=0.0009
θ=-0.7425, α=-3.1416, θ̇=-0.0023, α̇=0.0009
θ=-0.7426, α=-3.1416, θ̇=-0.0023, α̇=0.0010
θ=-0.7426, α=-3.1416, θ̇=-0.0023, α̇=0.0010
θ=-0.7426, α=-3.1416, θ̇=-0.0023, α̇=0.0010
θ=-0.7426, α=-3.1416, θ̇=-0.0023, α̇=0.0009
θ=-0.7427, α=-3.1416, θ̇=-0.0022, α̇=0.0009
θ=-0.7427, α=-3.1416, θ̇=-0.0022, α̇=0.0008
θ=-0.7427, α=-3.1416, θ̇=-0.0021, α̇=0.0007
θ=-0.7427, α=-3.1415, θ̇=-0.0020, α̇=0.0006
θ=-0.7427, α=-3.1415, θ̇=-0.0019, α̇=0.0004
θ=-0.7428, α=-3.1415, θ̇=-0.0019, α̇=0.0003
θ=-0.7428, α=-3.1415, θ̇=-0.0018, α̇=0.0001
θ=-0.7428, α=-3.1415, θ̇=-0.0017, α̇=-0.0000
θ=-0.7428, α=-3.1415, θ̇=-0.0016, α̇=-0.0002
θ=-0.7428, α=-3.1415, θ̇=-0.0015, α̇=-0.0003
θ=-0.7428, α=-3.1415, θ̇=-0.0014, α̇=-0.0005
θ=-0.7429, α=-3.1416, θ̇=-0.0014, α̇=-0.0006
θ=-0.7429, α=-3.1416, θ̇=-0.0013, α̇=-0.0007
θ=-0.7429, α=-3.1416, θ̇=-0.0012, α̇=-0.0008
θ=-0.7429, α=-3.1416, θ̇=-0.0012, α̇=-0.0008
θ=-0.7429, α=-3.1416, θ̇

In [256]:
@code_llvm forward_model_euler([0.0, 0.0, 0.1, 0.0], 0.0)


;  @ In[254]:2 within `forward_model_euler'
define nonnull %jl_value_t addrspace(10)* @julia_forward_model_euler_22014(%jl_value_t addrspace(10)* nonnull align 16 dereferenceable(40), double) {
top:
;  @ In[254]:4 within `forward_model_euler'
  %2 = call double @julia_normal_22008(double 8.400000e+00)
;  @ In[254]:5 within `forward_model_euler'
  %3 = call double @julia_normal_22008(double 4.200000e-02)
;  @ In[254]:6 within `forward_model_euler'
  %4 = call double @julia_normal_22008(double 4.200000e-02)
;  @ In[254]:7 within `forward_model_euler'
  %5 = call double @julia_normal_22008(double 9.500000e-02)
;  @ In[254]:8 within `forward_model_euler'
  %6 = call double @julia_normal_22008(double 8.500000e-02)
;  @ In[254]:10 within `forward_model_euler'
  %7 = call double @julia_normal_22008(double 2.700000e-04)
;  @ In[254]:11 within `forward_model_euler'
  %8 = call double @julia_normal_22008(double 2.400000e-02)
;  @ In[254]:12 within `forward_model_euler'
  %9 = call double @julia

In [257]:
f(x) = reduce(+, forward_model_euler(x, 0.0))

f (generic function with 1 method)

In [267]:
f([π, 0.1, 0., 0.])

-3.003726349432355

In [268]:
Zygote.refresh()

In [292]:
f'([0.01, 0.1, 0.1, 0.1])

4-element Array{Float64,1}:
 1.0               
 1.3339460527482354
 1.0002002442402704
 0.9998183999510273

In [270]:
g(x) = 5x + 3

g (generic function with 1 method)

In [271]:
g'(10)

5

In [ ]:
@code_llvm f'([0.0, 0.1, 0., 0.])